In [74]:
from __future__ import print_function
import importlib
from distutils.version import LooseVersion
import glob
import numpy as np
import os
import SimpleITK as sitk
import matplotlib.pyplot as plt
%matplotlib inline


In [106]:
import os
import SimpleITK as sitk
import matplotlib.pyplot as plt
%matplotlib inline

%run update_path_to_download_script
from downloaddata import fetch_data as fdata

from downloaddata import fetch_data_all
import numpy as np
inputDir = 'train/Amsterdam/101'
outputDir = 'output'

ERROR:root:File `'update_path_to_download_script.py'` not found.


In [75]:
def myshow(img, title=None, margin=0.05, dpi=80):
    nda = sitk.GetArrayViewFromImage(img)
    spacing = img.GetSpacing()
        
    if nda.ndim == 3:
        # fastest dim, either component or x
        c = nda.shape[-1]
        
        # the the number of components is 3 or 4 consider it an RGB image
        if not c in (3,4):
            nda = nda[nda.shape[0]//2,:,:]
    
    elif nda.ndim == 4:
        c = nda.shape[-1]
        
        if not c in (3,4):
            raise Runtime("Unable to show 3D-vector Image")
            
        # take a z-slice
        nda = nda[nda.shape[0]//2,:,:,:]
            
    ysize = nda.shape[0]
    xsize = nda.shape[1]
      
    # Make a figure big enough to accommodate an axis of xpixels by ypixels
    # as well as the ticklabels, etc...
    figsize = (1 + margin) * ysize / dpi, (1 + margin) * xsize / dpi

    fig = plt.figure(figsize=figsize, dpi=dpi)
    # Make the axis the right size...
    ax = fig.add_axes([margin, margin, 1 - 2*margin, 1 - 2*margin])
    
    extent = (0, xsize*spacing[1], ysize*spacing[0], 0)
    
    t = ax.imshow(nda,extent=extent,interpolation=None)
    
    if nda.ndim == 2:
        t.set_cmap("gray")
    
    if(title):
        plt.title(title)

In [76]:
help(np.pad)

Help on function pad in module numpy.lib.arraypad:

pad(array, pad_width, mode, **kwargs)
    Pads an array.
    
    Parameters
    ----------
    array : array_like of rank N
        Input array
    pad_width : {sequence, array_like, int}
        Number of values padded to the edges of each axis.
        ((before_1, after_1), ... (before_N, after_N)) unique pad widths
        for each axis.
        ((before, after),) yields same before and after pad for each axis.
        (pad,) or int is a shortcut for before = after = pad width for all
        axes.
    mode : str or function
        One of the following string values or a user supplied function.
    
        'constant'
            Pads with a constant value.
        'edge'
            Pads with the edge values of array.
        'linear_ramp'
            Pads with the linear ramp between end_value and the
            array edge value.
        'maximum'
            Pads with the maximum value of all or part of the
            vector

In [77]:
def loadData( rootPath, padding=0 ):
    imageFilenames = glob.glob(os.path.join(rootPath, "*", "pre", "T1.nii.gz"))      
    flairFilenames = glob.glob(os.path.join(rootPath, "*", "pre", "FLAIR.nii.gz"))
    labelFilenames = glob.glob(os.path.join(rootPath, "*", "wmh.nii.gz"))
    
    images = None # shape: (numImages, z, y, x, channels=1)
    labels = None
    masks  = None
    
    for imageFilename, flairFilename, labelFilename in zip(imageFilenames, flairFilenames, labelFilenames):
        # Load the images
        imageImage = sitk.ReadImage(imageFilename)
        flairImage = sitk.ReadImage(flairFilename)
        labelImage = sitk.ReadImage(labelFilename)
        
        # Convert to arrays
        imageArray = np.pad(sitk.GetArrayFromImage(imageImage), [(0,0),(padding,padding),(padding,padding)], 'constant')
        flairArray = np.pad(sitk.GetArrayFromImage(flairImage), [(0,0),(padding,padding),(padding,padding)], 'constant')
        labelArray = np.pad(sitk.GetArrayFromImage(labelImage), [(0,0),(padding,padding),(padding,padding)], 'constant')
        maskArray = labelArray > 0
        
        # Add to the images/labels array
        if images is None:
            images = imageArray.reshape([1] + list(imageArray.shape) + [1])
            images = np.concatenate([images, flairArray.reshape([1] + list(flairArray.shape) + [1])], axis=4)
            labels = labelArray.reshape([1] + list(labelArray.shape) + [1])
            masks  = maskArray.reshape([1] + list(maskArray.shape) + [1])
        else:
            tempArray = np.concatenate([imageArray.reshape([1] + list(imageArray.shape) + [1]), flairArray.reshape([1] + list(flairArray.shape) + [1])], axis=4)
            images = np.concatenate([images, tempArray])
            
            labels = np.concatenate([labels, labelArray.reshape([1] + list(labelArray.shape) + [1])])
            masks  = np.concatenate([masks, maskArray.reshape([1] + list(maskArray.shape) + [1])])
    
    size = flairImage.GetSize()
    print(size)
    nslices = 3
    slices = [ flairImage[:,:,s] for s in range(0, size[2], size[2]//10)]
    myshow(sitk.Tile(slices, [10,100]))                
    
    return images, labels, masks

In [78]:
rootPath = "train/Amsterdam/GE3T"
testPath = "train/Amsterdam/GE3T"
testImages, testLabels, testMasks = loadData(testPath)
testNonZeroIdx = np.nonzero(testMasks)

UnboundLocalError: local variable 'flairImage' referenced before assignment

In [ ]:
testImages.shape

In [ ]:
testLabels.shape

In [ ]:
testMasks.shape

In [79]:
type(testImages)

numpy.ndarray

In [80]:
import random
random.sample(range(20), 10)

[16, 0, 10, 14, 4, 3, 1, 11, 2, 9]

In [81]:
import os
path=os.path.dirname("train/Amsterdam/GE3T")
print(path)
print(os.path.basename(path))


train/Amsterdam
Amsterdam


In [82]:
listOfFiles = os.listdir("train/Utrecht")
result = [i for i in listOfFiles if not i.startswith('.')]
print(len(result))
print(result)
random.seed(88)
random.sample(result, 5)

FileNotFoundError: [Errno 2] No such file or directory: 'train/Utrecht'

In [83]:
def batchTestGeneratorMasked(images, labels, nonZeroIdx, shape):  
    idx = 0
    while idx < len(nonZeroIdx[0]):        
        batchImages = np.zeros(shape, dtype=np.float32)
        batchLabels = np.zeros((shape[0], 1), dtype=np.float32)
        
        for i in range(shape[0]):              
            zs = nonZeroIdx[1][idx]
            ze = zs + 1
            
            ys = int(nonZeroIdx[2][idx] - 0.5*(shape[1]-1))
            ye = int(nonZeroIdx[2][idx] + 0.5*(shape[1]-1) + 1)
            
            xs = int(nonZeroIdx[3][idx] - 0.5*(shape[2]-1))
            xe = int(nonZeroIdx[3][idx] + 0.5*(shape[2]-1) + 1)            
            
            batchImages[i,:,:,:] = images[nonZeroIdx[0][idx], zs:ze, ys:ye, xs:xe]
            batchLabels[i] = labels[nonZeroIdx[0][idx], nonZeroIdx[1][idx], nonZeroIdx[2][idx], nonZeroIdx[3][idx] ]
            
            idx += 1      
            
            # Check if we go beyond the image(s)
            if idx >= len(nonZeroIdx[0]):
                break
        
        # In case we have reached the last batch, crop to i+1                      
        yield batchImages[:i+1], batchLabels[:i+1]
    
    return

In [119]:
batchSize = 150
batchShape = (batchSize, 30, 30, 2)
iterator = batchTestGeneratorMasked(testImages, testLabels, testNonZeroIdx, batchShape)


In [120]:
print (type(iterator))

<class 'generator'>


In [121]:
def myshow(nda, title=None, margin=0.05, dpi=80):
    #nda = sitk.GetArrayViewFromImage(img)
    img = sitk.GetImageFromArray(nda)
    spacing = img.GetSpacing()
        
    if nda.ndim == 3:
        # fastest dim, either component or x
        c = nda.shape[-1]
        
        # the the number of components is 3 or 4 consider it an RGB image
        if not c in (3,4):
            nda = nda[nda.shape[0]//2,:,:]
    
    elif nda.ndim == 4:
        c = nda.shape[-1]
        
        
        if not c in (3,4):
            print("Unable to show 3D-vector Image")
            #raise Runtime("Unable to show 3D-vector Image")
            
        # take a z-slice
        nda = nda[nda.shape[0]//2,:,:,:]
            
    ysize = nda.shape[0]
    xsize = nda.shape[1]
      
    # Make a figure big enough to accommodate an axis of xpixels by ypixels
    # as well as the ticklabels, etc...
    figsize = (1 + margin) * ysize / dpi, (1 + margin) * xsize / dpi

    fig = plt.figure(figsize=figsize, dpi=dpi)
    # Make the axis the right size...
    ax = fig.add_axes([margin, margin, 1 - 2*margin, 1 - 2*margin])
    
    extent = (0, xsize*spacing[1], ysize*spacing[0], 0)
    
    t = ax.imshow(nda,extent=extent,interpolation=None)
    
    if nda.ndim == 2:
        t.set_cmap("gray")
    
    if(title):
        plt.title(title)

In [148]:
i = 0
result = None
novelty = None
for predictImages, predictLabels in iterator:
    print(predictImages.shape)
    img = sitk.GetImageFromArray(predictImages)
    size = img.GetSize()
    print(size)
    selectimage = [ img[:,:,s] for s in range(size[2]//8,size[2]-size[2]//8)]
    print(len(selectimage))
    slices2 = [selectimage[s] for s in range(0,len(selectimage),len(selectimage)//10)]
    print()
    myshow(sitk.Tile(slices2, [10,100]))
    resultModel = model.predict_on_batch(predictImages)
    resultNovelty = novelty_model.predict_on_batch(predictImages)
    
    if result is None:
        result = resultModel
        novelty = resultNovelty
    else:
        result = np.concatenate([result, resultModel])
        novelty = np.concatenate([novelty, resultNovelty])

#result = model.predict_generator(iterator, len(testNonZeroIdx[0]))


#%%
dst = np.zeros((1, 48, 240, 240, 1), dtype=np.int8)
dst[testMasks] = np.argmax(result, axis=1)
for k,i in enumerate(dst):
    a = sitk.GetImageFromArray(i[:,:,:,0])
    sitk.WriteImage(a,'X:/wml-distance/testing/'+str(k)+'.nii') 
    print(i[:,:,:,0].shape)

(150, 30, 30, 2)
(30, 30, 150)
114



RuntimeError: Exception thrown in SimpleITK Tile: /scratch/dashboard/SimpleITK-OSX10.6-x86_64-pkg/SimpleITK/Code/Common/include/sitkMemberFunctionFactory.hxx:209:
sitk::ERROR: Pixel type: vector of 32-bit float is not supported in 2D byN3itk6simple15TileImageFilterE

In [135]:
help(sitk.Tile)

Help on function Tile in module SimpleITK.SimpleITK:

Tile(*args)
    Tile(VectorOfImage images, VectorUInt32 layout, double defaultPixelValue=0.0) -> Image
    Tile(Image image1, VectorUInt32 layout, double defaultPixelValue=0.0) -> Image
    Tile(Image image1, Image image2, VectorUInt32 layout, double defaultPixelValue=0.0) -> Image
    Tile(Image image1, Image image2, Image image3, VectorUInt32 layout, double defaultPixelValue=0.0) -> Image
    Tile(Image image1, Image image2, Image image3, Image image4, VectorUInt32 layout, double defaultPixelValue=0.0) -> Image
    Tile(Image image1, Image image2, Image image3, Image image4, Image image5, VectorUInt32 layout, double defaultPixelValue=0.0) -> Image

